In [579]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)
library(ggplot2)
library(dummies)
library(stringr)
library(rms)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_score_finale.Rdata")
df <- df_score_finale
df <- subset(df,select= -c(degenzaPostTraOSP))

In [580]:
names(df) <- c('Centre', 'Gender', 'Age', 'LHOSpretx', 'Status', 'VaricealBleed', 'Ascites', 'Tips',  
                'PVthrombosis', 'NutritStatus', 'Previoussurg', 'Ventilationpretx', 'Vasopressors', 
               'InfusionAlb', 'PaO2FiO2', 'PlatCount', 'Albumin', 'Duration', 'PRBCintraop', 'FFPintraop', 'Plateletsintraop',
               'Fibrinogen', 'CRIO', 'Crystalloids', 'PPS5', 'Norepinend', 'Lactates', 'ColdIschemia', 'ReperfusSy', 'PCshunt',
               'Cavaveinanasth', 'VVBypass', 'Biliaryanasth', 'Openabdomen', 'Donorgender', 'Donortype', 'Organtype' ,'DonorECMO', 
               'RegionalPerf', 'OrganBiopsy', 'Machineperf', 'DonorSite', 'DonorCauseDeath', 'DonorAge', 'DonorHeight', 'DonorSodiemia',
               'DonorGGT', 'MELDNa', 'Alcoholic', 'cirrhosisDrugs', 'NASH', 'Lateretx', 'DonorMaastricht', 'DonorSteatosis' ,
               'HypoNormoter', 'Nocomorbodities', 'Hypertension', 'SeverePsychosis', 'Periodstx', 'Nighttx', 'Admsite' ,
               'Encephalopathy', 'Cholestatic', 'Rare', 'Viral', 'HCC', 'Respcomorbidities', 'CVcomorbidities', 'Neurocomorbidities',
               'Renalcomorbidities', 'Diabetes', 'Anticoagulation', 'MELDExcept', 'deg_cat')

In [563]:
library(Boruta)
# Decide if a variable is important or not using Boruta
boruta_output <- Boruta(deg_cat ~ ., data=df, doTrace=2)  # perform Boruta search


boruta_signif <- names(boruta_output$finalDecision[boruta_output$finalDecision %in% c("Confirmed", "Tentative")])  # collect Confirmed and Tentative variables
print(boruta_signif)  # significant variables


plot(boruta_output, cex.axis=1.2, las=2, xlab="", main="Variable Importance")  # plot variable importance


df_boruta <- df[,boruta_signif]

df_boruta <- apply(select_if(df,is.matrix),as.numeric)


 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

After 13 iterations, +53 secs: 

 confirmed 14 attributes: Albumin, Ascites, Centre, Duration, InfusionAlb and 9 more;

 rejected 38 attributes: Age, Alcoholic, Anticoagulation, Cavaveinanasth, Cholestatic and 33 more;

 still have 21 attributes left.


 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

After 18 iterations, +1 mins: 

 confirmed 4 attributes: CRIO, Crystalloids, FFPintraop, PPS5;

 rejected 7 attributes: Biliaryanasth, Donorgen

 [1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "CRIO"             "Crystalloids"     "PPS5"             "Norepinend"      
[17] "Lactates"         "ReperfusSy"       "Openabdomen"      "DonorSodiemia"   
[21] "MELDNa"           "Nocomorbodities"  "Admsite"          "Encephalopathy"  
[25] "HCC"             


ERROR: Error in match.fun(FUN): l'argomento "FUN"  non è specificato e non ha un valore predefinito


In [564]:
par(mar = c(13, 5, 2, 0.1))  
plot(boruta_output, cex.axis=1.3, las=2, xlab="", main="Variable Importance")

In [565]:
df_boruta$deg_cat <- df$deg_cat

In [566]:
boruta_signif #variabili selezionate

[1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "CRIO"             "Crystalloids"     "PPS5"             "Norepinend"      
[17] "Lactates"         "ReperfusSy"       "Openabdomen"      "DonorSodiemia"   
[21] "MELDNa"           "Nocomorbodities"  "Admsite"          "Encephalopathy"  
[25] "HCC"

In [581]:
df <- subset(df, select = c('Duration','Albumin','MELDNa','PRBCintraop','Lactates','deg_cat'))

In [582]:
df$deg_cat_bin = ifelse(df$deg_cat == 'fast',1,0)

In [584]:
df <- df[df$PRBCintraop < 52,] # 4 outliers

In [585]:
df$dataset <- sample(x=c("train","validate"),
size=nrow(df),
replace=TRUE,
prob=c(0.75,0.25))

## LOESS smoothing curves for numeric variables

In [586]:
df.cont<- df[df$dataset=="train",
c('deg_cat_bin','MELDNa','PRBCintraop','Lactates')]

#df.cat<- df[df$dataset=="train",
#c('deg_cat')]




ymark<-seq(0,1,0.1)

for(var in names(df.cont)[-1]){
xvar<-seq(min(df.cont[,var]),
max(df.cont[,var]),
length.out=10)
    
    
mypath <- file.path("C:/Users/abonini/Documents/score",
paste(paste("plot",var,sep = "_"),
"pdf", sep = "."))

pdf(file=mypath)

gg<-ggplot(df.cont, aes(x=df.cont[,var],y=deg_cat_bin))+
geom_jitter(size=1, alpha=0.2,height=0.05)+
stat_smooth(method="loess",colour="blue", size=1.5)+
xlab(var)+
ylab("Probability on length of hospital stay")+
theme_bw()+
geom_hline(yintercept=ymark,col="red")+
scale_y_continuous(breaks=ymark)+
geom_vline(xintercept=xvar,col="green",alpha=0.5)+
scale_x_continuous(breaks=round(xvar,2))
    
print(gg)
dev.off()
}

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'



## Choosing cut points and base reference

In [587]:
MELDNacut<-c(2.06,10,17.5,30)
MELDNab<-"[2.05,2.06]"

PRBCintraopcut <-c(0.1,3,7)
PRBCintraopb<-"[0,0.1]"

Lactatescut <- c(0.6,2.5,5)
Lactatesb <- "[0.5,0.6]"



cont.to.cat<-data.frame(id=c(1:nrow(df.cont)))

for (var in names(df.cont)[-1]) {
cat<-cut(df.cont[,var],
breaks=c(min(df.cont[,var]),
get(paste(var,"cut",sep="")),
max(df.cont[,var])),include.lowest= TRUE)
cat<-relevel(cat,ref=get(paste(var,"b",sep="")))
cont.to.cat<-cbind(cont.to.cat,cat)
}

## Calculating scores for each level

In [588]:
df.cont.to.cat<-cont.to.cat[,-1]

names(df.cont.to.cat)<-names(df.cont)[-1]

df.final<-cbind(df.cont.to.cat,

deg_cat_bin=df.cont$deg_cat_bin)

mod<-glm(deg_cat_bin~.,
df.final, family="binomial")
score<-round(coef(mod)[-1])


score.cont<-score


## Calculating scores for numeric variables

In [589]:
var.cont<-as.character(1)

for(var in names(score.cont)){
var.red<-sub("(\\(|\\[)[0-9]+.*", "", var)
var.cont<-c(var.cont,var.red)
}

var.cont<-unique(var.cont)[-1]

for(var in var.cont){
df[,paste(var,"points",sep=".")]<-as.numeric(NA)
}

for (var in names(score.cont)){
var.red<-sub("(\\(|\\[)[0-9]+.*", "", var)
var.low<-as.numeric(str_extract(var,'(?<=(\\(|\\[))[0-9]+\\.*[0-9]*(?=\\,)'))
var.upper<-as.numeric(str_extract(var,'(?<=\\,)[0-9]+\\.*[0-9]*(?=\\])'))
df[,paste(var.red,"points",sep=".")]<-ifelse(
df[,var.red]<=var.upper&df[,var.red]>=var.low,
score[var],df[,paste(var.red,"points",sep=".")])
}

for(var in var.cont){
df[,paste(var,"points",sep=".")]<-ifelse(
is.na(df[,paste(var,"points",sep=".")]),
0,df[,paste(var,"points",sep=".")])
}


# Calculate scores for factor variables and take a sum

In [590]:
var.cat<-names(df.cat)

for(var in var.cat){
df[,paste(var,"points",sep=".")]<-as.numeric(NA)
}

for (var in var.cat){
score.var<-score.cat[grep(var,names(score.cat))]
names(score.var)<-sub(var,"",names(score.var))
for(i in 1:(length(levels(df[,var]))-1)){
df[,paste(var,"points",
sep=".")]<-ifelse(
df[,var]==names(score.var)[i]&
is.na(df[,paste(var,"points",sep=".")]),
score.var[i],
df[,paste(var,"points",sep=".")])}
}

for(var in var.cat){
df[,paste(var,"points",sep=".")]<-ifelse(
is.na(df[,paste(var,"points",sep=".")]),
0,df[,paste(var,"points",sep=".")])}


In [591]:
df$score<-rowSums(df[,grepl("\\.+points",names(df))])

In [592]:
head(df)

,Duration,Albumin,MELDNa,PRBCintraop,Lactates,deg_cat,deg_cat_bin,dataset,MELDNa.points,PRBCintraop.points,Lactates.points,score
15,347,4.0,10.26,5,2.3,fast,1,train,12,0,-1,11
16,361,3.0,26.68,44,2.4,NotFast,0,train,11,-1,-1,9
29,261,4.3,4.44,0,2.8,fast,1,train,13,0,-1,12
30,370,2.7,25.78,14,2.5,NotFast,0,validate,11,-1,-1,9
32,314,4.5,4.14,0,1.7,fast,1,train,13,0,-1,12
33,317,4.5,7.29,1,1.6,fast,1,validate,13,0,-1,12


In [593]:
score

MELDNa(2.06,10]     MELDNa(10,17.5]     MELDNa(17.5,30]     MELDNa(30,48.3] 
                 13                  12                  11                  10 
 PRBCintraop(0.1,3]    PRBCintraop(3,7] PRBCintraop(7,45.3]   Lactates(0.6,2.5] 
                  0                   0                  -1                  -1 
    Lactates(2.5,5]      Lactates(5,28] 
                 -1                  -2

## Converting the score to probability of mortality and comparing it to observed number of deaths

In [594]:
glmod<-glm(deg_cat_bin~score,
df[df$dataset=="train",],
family="binomial")

newx<-seq(min(df[df$dataset=="train",]$score),
max(df[df$dataset=="train",]$score))

prd<-predict(glmod,
newdata=data.frame(score=newx),
type="response",
se.fit=T)

count<-as.matrix(table(cut(df[df$dataset=="train",]$score,
breaks=seq(min(df[df$dataset=="train",]$score),
max(df[df$dataset=="train",]$score)),
include.lowest = T),
df[df$dataset=="train",]$deg_cat_bin))


## Visualization of the relationship between scores and probability of outcome events

In [595]:
par(mar=c(5,4,4,5)+.1)

barplot(t(count),
main="Scores versus probability on length of hospital stay",
xlab="Scores",
ylab="Observed number of patients",
space=0,
col=c("yellow","lightblue"))

legend("topleft",fill=c("yellow","lightblue",NA),
lty = c(NA,NA,1),lwd=c(NA,NA,2),
legend=c("fast","NotFast",
"Predicted Prob."),
col=c("black"),
border = c("black","black",NA))

par(new=TRUE)

plot(prd$fit~newx,
type="l",col="black",
lwd=2,xaxt="n",yaxt="n",
xlab="",ylab="")

polygon(c(rev(newx), newx),
c(rev(prd$fit+1.96*prd$se.fit),
prd$fit-1.96*prd$se.fit),
col = adjustcolor('grey80',alpha=0.5),
border = NA)

lines(newx, prd$fit+1.96*prd$se.fit,
lty = 'dashed', col = 'red')

lines(newx, prd$fit-1.96*prd$se.fit,
lty = 'dashed', col = 'red')

axis(4)

mtext("Predicted length of hospital stay",
side=4,line=3)


## Validation of the score

In [596]:
ddist <- datadist(df)

options(datadist='ddist')

f.score<-lrm(deg_cat_bin~score,
df[df$dataset=="train",],
x=TRUE,y=TRUE)

phat.score<-predict(f.score,
df[df$dataset=="validate",],
type="fitted")

v.score<-val.prob(phat.score,
df[df$dataset=="validate",]$deg_cat_bin,
m=20)


Warning message in min(xx[xx > upper]):
"no non-missing arguments to min; returning Inf"
